# **EIASR 21Z - Face Mask Detection**

We used this [project](https://www.kaggle.com/meghanatiyyali560/sailaja-meghana-project) as a guide how to use annotations provided with [image dataset we used](https://www.kaggle.com/andrewmvd/face-mask-detection). 

## **1. Environment preparation**

### Imports

In [ ]:
import os
import glob
import cv2
import shutil
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from keras_preprocessing.image import ImageDataGenerator
from xml.etree import ElementTree
from pandas import DataFrame

### Constants

In [ ]:
path = 'drive/MyDrive/datasets/face-mask-detection'

BATCH_SIZE = 32
SEED_SIZE = 42
EPOCHS = 50

key_cropped = 'cropped_image_file'
key_label = 'label'
key_file = 'file'
key_image_file = 'image_file'
key_annotation = 'annotation_file'

In [ ]:
annotations_dir = path + '/annotations'
images_dir = path + '/images'
temp_dir = path + '/temp'
cropped_dir = temp_dir + '/cropped_images'

classifier_path = "drive/MyDrive/models/classifier.h5"

os.mkdir(temp_dir)
os.mkdir(cropped_dir)

FileExistsError: ignored

In [ ]:
!ls 'drive/MyDrive/datasets/face-mask-detection'
!ls 'drive/MyDrive/datasets/face-mask-detection/temp'

annotations  images  temp
cropped_images


## **2. Test dataset preparation**

In [ ]:
data = {'xmin': [], 'ymin': [], 'xmax': [], 'ymax': [], 
        'label': [], 'file': [], 'width': [], 'height': []}

for annotation in glob.glob(annotations_dir + '/*.xml'):
    tree = ElementTree.parse(annotation)
    
    for element in tree.iter():
        if 'size' in element.tag:
            for attribute in list(element):
                if 'width' in attribute.tag:
                    width = int(round(float(attribute.text)))
                if 'height' in attribute.tag:
                    height = int(round(float(attribute.text)))

        if 'object' in element.tag:
            for attribute in list(element):
                if 'name' in attribute.tag:
                    name = attribute.text
                    data['label'] += [name]
                    data['width'] += [width]
                    data['height'] += [height]
                    data['file'] += [annotation.split('/')[-1][0:-4]]
                            
                if 'bndbox' in attribute.tag:
                    for dimension in list(attribute):
                        if 'xmin' in dimension.tag:
                            xmin = int(round(float(dimension.text)))
                            data['xmin'] += [xmin]
                        if 'ymin' in dimension.tag:
                            ymin = int(round(float(dimension.text)))
                            data['ymin'] += [ymin]                                
                        if 'xmax' in dimension.tag:
                            xmax = int(round(float(dimension.text)))
                            data['xmax'] += [xmax]                                
                        if 'ymax' in dimension.tag:
                            ymax = int(round(float(dimension.text)))
                            data['ymax'] += [ymax]


### Cropping images

In [ ]:
df = DataFrame(data)
df[key_annotation] = df[key_file] + '.xml'
df[key_image_file] = df[key_file] + '.png'
df[key_cropped] = df[key_file]

df

,xmin,ymin,xmax,ymax,label,file,width,height,annotation_file,image_file,cropped_image_file
0,23,114,77,172,mask_weared_incorrect,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11
1,147,157,200,211,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11
2,201,191,230,234,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11
3,243,192,293,247,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11
4,309,182,346,224,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11
...,...,...,...,...,...,...,...,...,...,...,...
4067,247,78,285,107,without_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703
4068,312,82,331,112,with_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703
4069,331,93,371,135,with_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703
4070,384,116,400,156,without_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703


In [ ]:
def crop(write=False):
  for i in range(len(df)):
      image_path = images_dir + '/' + df[key_image_file].iloc[i]
      
      df[key_cropped].iloc[i] = df[key_cropped].iloc[i] + '-' + str(i) + '.png'
      cropped_image_filename = df[key_cropped].iloc[i]
      
      xmin = df['xmin'].iloc[i]
      ymin = df['ymin'].iloc[i]
      xmax = df['xmax'].iloc[i]
      ymax = df['ymax'].iloc[i]
      
      if write:
        image = cv2.imread(image_path)
        cropped_image = image[ymin:ymax, xmin:xmax]
        cropped_image_dir = os.path.join(cropped_dir + '/', cropped_image_filename) 
        cv2.imwrite(cropped_image_dir, cropped_image)

In [ ]:
crop()

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
test_df = df[:800]
train_df = df[800:]

classes = list(train_df[key_label].unique())

df

,xmin,ymin,xmax,ymax,label,file,width,height,annotation_file,image_file,cropped_image_file
0,23,114,77,172,mask_weared_incorrect,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11-0.png
1,147,157,200,211,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11-1.png
2,201,191,230,234,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11-2.png
3,243,192,293,247,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11-3.png
4,309,182,346,224,with_mask,maksssksksss11,400,267,maksssksksss11.xml,maksssksksss11.png,maksssksksss11-4.png
...,...,...,...,...,...,...,...,...,...,...,...
4067,247,78,285,107,without_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703-4067.png
4068,312,82,331,112,with_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703-4068.png
4069,331,93,371,135,with_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703-4069.png
4070,384,116,400,156,without_mask,maksssksksss703,400,267,maksssksksss703.xml,maksssksksss703.png,maksssksksss703-4070.png


In [ ]:
sorted_label_df = DataFrame(train_df['label'].value_counts()).reset_index()
sorted_label_df.rename(columns = {'index': 'label', 'label': 'count'}, inplace = True)
sorted_label_df

,label,count
0,with_mask,2591
1,without_mask,580
2,mask_weared_incorrect,101


In [ ]:
image_width = []
image_height = []

for i in range(len(train_df)):
    cropped_image_path = cropped_dir + '/' + train_df[key_cropped].iloc[i]
    cropped_image = cv2.imread(cropped_image_path)
    image_width.append(cropped_image.shape[0])
    image_height.append(cropped_image.shape[1])

In [ ]:
image_target_size = (int(np.median(image_width)), int(np.median(image_height)))
image_target_size

(24, 22)

### // TODO: Data Augmentation

In [ ]:
data_augmentation = tf.keras.Sequential([
  tf.keras.layers.RandomFlip('horizontal'),
  tf.keras.layers.RandomRotation(0.2),
])

for image, _ in train_dataset.take(1):
  plt.figure(figsize=(10, 10))
  first_image = image[0]
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    augmented_image = data_augmentation(tf.expand_dims(first_image, 0))
    plt.imshow(augmented_image[0] / 255)
    plt.axis('off')

### Loading datasets

In [ ]:
training_image_generator = ImageDataGenerator(rescale = 1. / 255., validation_split = 0.25)

training_generator = training_image_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = cropped_dir,
    x_col = key_cropped,
    y_col = key_label,
    subset = 'training',
    batch_size = BATCH_SIZE,
    seed = SEED_SIZE,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

validation_generator = training_image_generator.flow_from_dataframe(
    dataframe = train_df,
    directory = cropped_dir,
    x_col = key_cropped,
    y_col = key_label,
    subset = 'validation',
    batch_size = BATCH_SIZE,
    seed = SEED_SIZE,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

Found 2454 validated image filenames belonging to 3 classes.
Found 818 validated image filenames belonging to 3 classes.


In [ ]:
test_image_generator = ImageDataGenerator(rescale = 1. / 255.)

test_generator = test_image_generator.flow_from_dataframe(
    dataframe = test_df,
    directory = cropped_dir,
    x_col = key_cropped,
    y_col = key_label,
    batch_size = BATCH_SIZE,
    seed = SEED_SIZE,
    shuffle = True,
    class_mode = 'categorical',
    target_size = image_target_size
)

Found 800 validated image filenames belonging to 3 classes.


In [ ]:
input_shap = [int(np.median(image_width)), int(np.median(image_height)), 3]

## **3. Model preparation for transfer learning**

### Preparing ResNet50 model

In [ ]:
base_input_shape = (250, 250, 3)
weights = 'imagenet'

# without dense part
base_model = ResNet50(weights=weights, 
                      include_top=False, 
                      input_shape=base_input_shape)

94781440/94765736 [==============================] - 1s 0us/step


### Freezing base layers

In [ ]:
for layer in base_model.layers:
    layer.trainable = False

### Adding Global Avg Pooling 2D and Dense layer

In [ ]:
num_classes = 3
activation = 'sigmoid' #'relu'
learning_rate = 0.1 #0.01 - 0.9


global_avg_pooling = keras.layers.GlobalAveragePooling2D()(base_model.output)
output = keras.layers.Dense(num_classes, activation=activation)(global_avg_pooling)

classifier = keras.models.Model(inputs=base_model.input,
                                outputs=output,
                                name='ResNet50')

classifier.summary()

Model: "ResNet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 250, 250, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 256, 256, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 125, 125, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

### Enable mechanisms preventing overlearning

In [ ]:
patience = 3

earlystop = EarlyStopping(monitor='val_loss',
                          restore_best_weights=True,
                          patience=patience,
                          verbose=1)

### Enable mechanisms preventing interrupting learning process

In [ ]:
checkpoint = ModelCheckpoint(classifier_path,
                             monitor="val_loss",
                             mode="min",
                             save_best_only=True,
                             verbose=1)

callbacks = [earlystop, checkpoint]

### Define training

In [ ]:
def train(learning_rate):
  classifier.compile(loss='categorical_crossentropy',
                    optimizer=Adam(learning_rate=learning_rate),
                    metrics=['accuracy'])

  classifier.fit(training_generator,
                epochs=EPOCHS,
                steps_per_epoch=len(training_generator),
                callbacks=callbacks,
                validation_data=validation_generator,
                validation_steps=len(validation_generator))

  classifier.save(classifier_path)

### Start training process (activation function: sigmoid)

In [ ]:
classifier.compile(loss='categorical_crossentropy',
                   optimizer=Adam(learning_rate=0.01),
                   metrics=['accuracy'])

classifier.fit(training_generator,
               epochs=EPOCHS,
               steps_per_epoch=len(training_generator),
               callbacks=callbacks,
               validation_data=validation_generator,
               validation_steps=len(validation_generator))

classifier.save(classifier_path)

Epoch 1/50
77/77 [==============================] - ETA: 0s - loss: 0.6745 - accuracy: 0.7787
Epoch 00001: val_loss improved from inf to 0.47272, saving model to drive/MyDrive/models/classifier.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


77/77 [==============================] - 18s 203ms/step - loss: 0.6745 - accuracy: 0.7787 - val_loss: 0.4727 - val_accuracy: 0.7861
Epoch 2/50
77/77 [==============================] - ETA: 0s - loss: 0.4697 - accuracy: 0.8097
Epoch 00002: val_loss did not improve from 0.47272
77/77 [==============================] - 13s 170ms/step - loss: 0.4697 - accuracy: 0.8097 - val_loss: 0.5255 - val_accuracy: 0.7738
Epoch 3/50
77/77 [==============================] - ETA: 0s - loss: 0.4515 - accuracy: 0.8187
Epoch 00003: val_loss improved from 0.47272 to 0.46526, saving model to drive/MyDrive/models/classifier.h5
77/77 [==============================] - 15s 199ms/step - loss: 0.4515 - accuracy: 0.8187 - val_loss: 0.4653 - val_accuracy: 0.8117
Epoch 4/50
77/77 [==============================] - ETA: 0s - loss: 0.4492 - accuracy: 0.8219
Epoch 00004: val_loss improved from 0.46526 to 0.44265, saving model to drive/MyDrive/models/classifier.h5
77/77 [==============================] - 16s 203ms/step -

1st try (sigmoid): 

- epochs: 12
- from loss: 0.6745 - accuracy: 0.7787 
- to loss: 0.4407 - accuracy: 0.8399 - val_loss: 0.4332 - val_accuracy: 0.8325

### Start training process (activation function: ReLU)

In [ ]:
train()

Epoch 1/50
77/77 [==============================] - ETA: 0s - loss: 3.2512 - accuracy: 0.7983
Epoch 00001: val_loss improved from inf to 3.66499, saving model to drive/MyDrive/models/classifier.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


77/77 [==============================] - 27s 296ms/step - loss: 3.2512 - accuracy: 0.7983 - val_loss: 3.6650 - val_accuracy: 0.7726
Epoch 2/50
77/77 [==============================] - ETA: 0s - loss: 3.2512 - accuracy: 0.7983
Epoch 00002: val_loss did not improve from 3.66499
77/77 [==============================] - 16s 210ms/step - loss: 3.2512 - accuracy: 0.7983 - val_loss: 3.6650 - val_accuracy: 0.7726
Epoch 3/50
77/77 [==============================] - ETA: 0s - loss: 3.2512 - accuracy: 0.7983
Epoch 00003: val_loss did not improve from 3.66499
77/77 [==============================] - 16s 206ms/step - loss: 3.2512 - accuracy: 0.7983 - val_loss: 3.6650 - val_accuracy: 0.7726
Epoch 4/50
77/77 [==============================] - ETA: 0s - loss: 3.2512 - accuracy: 0.7983Restoring model weights from the end of the best epoch: 1.

Epoch 00004: val_loss did not improve from 3.66499
77/77 [==============================] - 16s 212ms/step - loss: 3.2512 - accuracy: 0.7983 - val_loss: 3.6650 -

2st try (ReLU): 

- epochs: 4
- from loss: 3.2512 - accuracy: 0.7983 
- to loss: 3.2512 - accuracy: 0.7983 - val_loss: 3.6650 - val_accuracy: 0.7726

### Start training process (learning rate 0.01 to 0.8)

In [ ]:
train(0.1)

Epoch 1/50
77/77 [==============================] - ETA: 0s - loss: 5.3816 - accuracy: 0.7319
Epoch 00001: val_loss improved from inf to 2.11751, saving model to drive/MyDrive/models/classifier.h5


/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


77/77 [==============================] - 23s 254ms/step - loss: 5.3816 - accuracy: 0.7319 - val_loss: 2.1175 - val_accuracy: 0.8154
Epoch 2/50
77/77 [==============================] - ETA: 0s - loss: 1.5832 - accuracy: 0.7861
Epoch 00002: val_loss improved from 2.11751 to 0.50481, saving model to drive/MyDrive/models/classifier.h5
77/77 [==============================] - 18s 233ms/step - loss: 1.5832 - accuracy: 0.7861 - val_loss: 0.5048 - val_accuracy: 0.8301
Epoch 3/50
77/77 [==============================] - ETA: 0s - loss: 1.1814 - accuracy: 0.7844
Epoch 00003: val_loss did not improve from 0.50481
77/77 [==============================] - 17s 224ms/step - loss: 1.1814 - accuracy: 0.7844 - val_loss: 0.8854 - val_accuracy: 0.8423
Epoch 4/50
77/77 [==============================] - ETA: 0s - loss: 0.6803 - accuracy: 0.8199
Epoch 00004: val_loss did not improve from 0.50481
77/77 [==============================] - 17s 216ms/step - loss: 0.6803 - accuracy: 0.8199 - val_loss: 0.6645 - va

In [ ]:
train(0.5)

Epoch 1/50
77/77 [==============================] - ETA: 0s - loss: 13.9180 - accuracy: 0.7294
Epoch 00001: val_loss did not improve from 0.50481
77/77 [==============================] - 21s 229ms/step - loss: 13.9180 - accuracy: 0.7294 - val_loss: 7.1396 - val_accuracy: 0.7848
Epoch 2/50
77/77 [==============================] - ETA: 0s - loss: 8.8763 - accuracy: 0.7677
Epoch 00002: val_loss did not improve from 0.50481
77/77 [==============================] - 16s 208ms/step - loss: 8.8763 - accuracy: 0.7677 - val_loss: 4.4498 - val_accuracy: 0.8545
Epoch 3/50
77/77 [==============================] - ETA: 0s - loss: 5.0654 - accuracy: 0.7991
Epoch 00003: val_loss did not improve from 0.50481
77/77 [==============================] - 16s 203ms/step - loss: 5.0654 - accuracy: 0.7991 - val_loss: 9.1293 - val_accuracy: 0.7946
Epoch 4/50
77/77 [==============================] - ETA: 0s - loss: 7.5359 - accuracy: 0.7673
Epoch 00004: val_loss did not improve from 0.50481
77/77 [===============

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


In [ ]:
train(0.8)

Epoch 1/50
77/77 [==============================] - ETA: 0s - loss: 29.5014 - accuracy: 0.7832
Epoch 00001: val_loss did not improve from 0.50481
77/77 [==============================] - 21s 227ms/step - loss: 29.5014 - accuracy: 0.7832 - val_loss: 13.2771 - val_accuracy: 0.6210
Epoch 2/50
77/77 [==============================] - ETA: 0s - loss: 12.8165 - accuracy: 0.7857
Epoch 00002: val_loss did not improve from 0.50481
77/77 [==============================] - 16s 208ms/step - loss: 12.8165 - accuracy: 0.7857 - val_loss: 12.5386 - val_accuracy: 0.8753
Epoch 3/50
77/77 [==============================] - ETA: 0s - loss: 10.6897 - accuracy: 0.8040
Epoch 00003: val_loss did not improve from 0.50481
77/77 [==============================] - 16s 203ms/step - loss: 10.6897 - accuracy: 0.8040 - val_loss: 13.4329 - val_accuracy: 0.7726
Epoch 4/50
77/77 [==============================] - ETA: 0s - loss: 10.3612 - accuracy: 0.7954
Epoch 00004: val_loss did not improve from 0.50481
77/77 [=======

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


## **4. Cleanup**

In [ ]:
shutil.rmtree(temp_dir)

FileNotFoundError: ignored

In [ ]:
!ls 'drive/MyDrive/datasets/face-mask-detection'

annotations  images
